# This file is for generating different confusion matricies!

In [10]:
# Import libraries
import sys
sys.path.insert(0, '/yunity/arusty/PF-GAP/')

from experiments.model_helpers import *
import pandas as pd


#* Load in the data
static2024 = pd.read_csv('~/PF-GAP/data/static2024.csv')
static2023 = pd.read_csv('~/PF-GAP/data/static2023.csv')
static2022 = pd.read_csv('~/PF-GAP/data/static2022.csv')
static2025 = pd.read_csv('~/PF-GAP/data/static2025.csv')
time_series = np.array(pd.read_csv('~/PF-GAP/data/time_series.csv'))
labels = pd.read_csv('~/PF-GAP/data/labels.csv')
labels = np.array(labels).flatten()

time_series_hourly = pd.read_csv("~/PF-GAP/data/HourlyReturnsSingleYear.csv")
labels_hourly = time_series_hourly.pop("Sector").values
tickers = time_series_hourly.pop("Ticker").values


# Hourly Data
static_data = pd.read_csv("~/PF-GAP/data/sp500_fundamentals_combined.csv")
static_hourly = static_data[static_data['Year'] == 2024].reset_index(drop=True)
static_hourly = static_hourly.set_index('Ticker').reindex(tickers).reset_index().drop(columns=["Ticker", "Year"])
if static_hourly.isnull().values.any():
    static_hourly = static_hourly.fillna(static_hourly.median(numeric_only=True))

print("-- Data Loaded --")

-- Data Loaded --


In [ ]:
# TODO: Build Confusion matrix function


# TODO: Get the results to compare the hourly 